In [50]:
from sklearn.datasets import load_digits
digits = load_digits(as_frame=True)

x_data = digits.data
y_data = digits.target

digits['data'].shape, digits['target'].shape

((1797, 64), (1797,))

In [58]:
import pandas as pd
digits = pd.concat([x_data, y_data],axis=1)

In [59]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1,10),
}

gs = GridSearchCV(model, params).fit(digits.iloc[:, :-1], digits.iloc[:, -1])

In [60]:
report = pd.DataFrame(gs.cv_results_)
report

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002887,0.002083,0.019148,0.007954,1,{'n_neighbors': 1},0.961111,0.952778,0.966574,0.986072,0.955432,0.964393,0.011838,3
1,0.003754,0.006556,0.029571,0.006343,2,{'n_neighbors': 2},0.961111,0.966667,0.969359,0.977716,0.961003,0.967171,0.006181,1
2,0.003151,0.006302,0.023216,0.007231,3,{'n_neighbors': 3},0.955556,0.958333,0.966574,0.986072,0.966574,0.966622,0.010672,2
3,0.003124,0.006248,0.027402,0.007324,4,{'n_neighbors': 4},0.947222,0.958333,0.966574,0.980501,0.966574,0.963841,0.010946,4
4,0.005137,0.005305,0.025638,0.005471,5,{'n_neighbors': 5},0.947222,0.955556,0.966574,0.980501,0.963788,0.962728,0.011169,5
5,0.002792,0.000977,0.027674,0.001757,6,{'n_neighbors': 6},0.944444,0.958333,0.966574,0.974930,0.952646,0.959386,0.010612,7
6,0.002681,0.001391,0.026633,0.004349,7,{'n_neighbors': 7},0.936111,0.961111,0.969359,0.980501,0.952646,0.959946,0.015059,6
7,0.004533,0.002105,0.027165,0.003968,8,{'n_neighbors': 8},0.936111,0.958333,0.969359,0.977716,0.949861,0.958276,0.014580,8
8,0.003452,0.002229,0.028435,0.001328,9,{'n_neighbors': 9},0.930556,0.952778,0.972145,0.977716,0.949861,0.956611,0.016887,9


In [61]:
print( gs.best_score_ )
print( gs.best_params_ )
print( gs.best_estimator_ )

0.9671711544413494
{'n_neighbors': 2}
KNeighborsClassifier(n_neighbors=2)


In [62]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, digits.iloc[:, :-1], digits.iloc[:, -1],cv=5)

array([0.94722222, 0.95555556, 0.96657382, 0.98050139, 0.9637883 ])

In [63]:
from sklearn.model_selection import cross_val_predict
cross_val_predict(model,digits.iloc[:, :-1], digits.iloc[:, -1], cv=5)

array([0, 1, 8, ..., 8, 9, 8])

In [53]:
# !pip install xgboost

In [65]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

boston = load_boston()

x_train, x_test, y_train, y_test = train_test_split(boston['data'],
                                                   boston['target'],
                                                   random_state=0)
model = xgb.XGBRegressor(objective='reg:linear')
model.fit(x_train, y_train)

p_train = model.predict(x_train)
p_test = model.predict(x_test)

r2_score(y_train, p_train), r2_score(y_test, p_test)

[13:48:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


(0.999999003030776, 0.7476326752660457)

In [66]:
# !pip install lightgbm

In [67]:
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

boston = load_boston()

x_train, x_test, y_train, y_test = train_test_split(boston['data'],
                                                   boston['target'],
                                                   random_state=0)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
    'objective':'regression',
}

model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                 early_stopping_rounds=5)

p_train = model.predict(x_train, num_iteration=model.best_iteration)
p_test = model.predict(x_test, num_iteration=model.best_iteration)

r2_score(y_train, p_train), r2_score(y_test, p_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000221 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 973
[LightGBM] [Info] Number of data points in the train set: 379, number of used features: 13
[LightGBM] [Info] Start training from score 22.608707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 70.5396
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 61.4369
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 53.753
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 47.6561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l2: 42.6525
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's l2: 

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


(0.9524522112992748, 0.7369037475494296)